In [1]:
import re
from itertools import chain
from tqdm import tqdm
import numpy as np
import collections

DICT = []

STATE_WORD = 0
STATE_TYPE = 1
STATE_MEANING_START = 2

class DictProcessor():
    def __init__(self):
        self.word = ""
        self.meaning = ""
        
        self.state = STATE_WORD
        
        self.dict = []

    def update(self, line):
        if line == "":
            self.go(STATE_WORD)
            return    
        elif re.match(r"^\s*\d+\.(.*)", line):
            m = re.match(r"^\s*\d+\.(.*)", line)
            line = m.group(1)
            self.go(STATE_TYPE)
        elif re.match(r"^\s*\d+\)", line):   
            m = re.match(r"^\s*\d+\)(.*)", line)
            line = m.group(1)
            self.go(STATE_MEANING_START)
        
        self._process(line)
        
    def _process(self, line):
        if self.state == STATE_WORD:
            self.word = line
            self.go(STATE_TYPE)
        elif self.state == STATE_TYPE:
            pass
            self.go(STATE_MEANING_START)
        elif self.state == STATE_MEANING_START:
            self.meaning += " " + line.strip()
    
    def _publish(self):
        self.meaning = self.meaning.strip()
        if self.word != "" and self.meaning != "" and len(self.word) >= 3:
            if "см. " in self.meaning or ":" in self.meaning or "*" in self.meaning: 
                pass
            else:
                self.dict.append((self.word, self.meaning))
            
            #print("!!!| [%s] - [%s]" % (self.word, self.meaning))        
        self.meaning = ""                        
    
    def go(self, new_state):
        if self.state == STATE_MEANING_START:
            self._publish()
        
        self.state = new_state   
        
    def post_process(self):
        counts = collections.Counter((w_m[0] for w_m in self.dict))
        
        self.dict = list(filter(lambda w_m: counts[w_m[0]] == 1, self.dict))        

processor = DictProcessor()        
        
with open("efremova.txt") as fin:
    row = -1
    
    state = STATE_WORD
    
    word = ""
    for line in fin:
        row += 1
        
        if row < 3:
            continue
            
        if row > 100 and False:
            break
        
        line = line.strip()                
        
        #print("%d: %s" % (processor.state, line))
        
        processor.update(line)                        
        
processor.post_process()        

In [2]:
print(len(processor.dict))

49674


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
NGRAMS = (2,4)
words_tr = TfidfVectorizer(analyzer='char', ngram_range=NGRAMS)
f_words = words_tr.fit_transform((w_m[0] for w_m in processor.dict))

In [5]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
cache = {}

def preprocess(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0].normal_form
        
    return cache[word]    

def tokenizer(line):
    res = []
    for w in re.findall("\w+", line):
        res.append(preprocess(w))
        
    return res    

meaning_tr = TfidfVectorizer(tokenizer=tokenizer)
f_meaning = meaning_tr.fit_transform((w_m[1] for w_m in processor.dict))

In [6]:
for i, (k, v) in enumerate(cache.items()):
    print("%s-%s" % (k, v))
    if i > 20:
        break

старинная-старинный
мелкая-мелкий
серебряная-серебряный
монета-монета
имевшая-иметь
хождение-хождение
на-на
кавказе-кавказ
а-а
также-также
аббатисса-аббатисса
настоятельница-настоятельница
католического-католический
женского-женский
монастыря-монастырь
слово-слово
образованное-образовать
из-из
первых-один
букв-буква
единиц-единица
в-в


In [7]:
for i, w in enumerate(meaning_tr.get_feature_names()):
    if i > 100:
        print(w)
    if i > 300:
        break

206
21
212
22
23
24
2400
244
25
2500
27
28
2а1
2а2
3
30
300
3000
32
335
34
35
36
366
4
40
400
4047
44
45
450
46
5
50
500
58
6
60
6000
7
70
79
8
80
81
83
86
9
90
91
b
c
i
ii
iii
iv
ix
j
p
pluralia
s
tantum
v
vi
vii
viii
x
xi
xii
xiii
xiv
xix
xv
xvi
xvii
xviii
xx
а
абак
аббатисса
аббревиатура
абвер
абзац
абиссаль
абиссинец
абиссиния
аболиционизм
абонемент
абориген
абразив
абразивный
абрикос
абрикосовый
абрис
абсолют
абсолютизм
абсолютно
абсолютный
абсорбция
абстрактный
абстракционизм
абстракция
абсурд
абсцесс
авангардизм
авангардистский
аванс
авансом
авантюра
авантюрист
авар
авария
август
авенариус
авиабензин
авиабомба
авиадвигатель
авиадесант
авиазавод
авиакомпания
авиалиния
авиамодель
авиамодельный
авиамотор
авианосец
авиапассажир
авиапочта
авиаприбор
авиапромышленность
авиаразведка
авиасвязь
авиасклад
авиастроение
авиатор
авиатранспорт
авиатрасса
авиаучилище
авиационный
авиация
авиачасть
авитаминоз
аврал
австерия
австралийский
австралия
австриец
австрия
австро
авто
автобиография
автоб

In [8]:
print(f_meaning.shape)

(49674, 33414)


In [9]:
print(f_meaning[2, :])

  (0, 26635)	0.427375860879
  (0, 16203)	0.137861447057
  (0, 8395)	0.17046590785
  (0, 16552)	0.185800592885
  (0, 2038)	0.146346995168
  (0, 6655)	0.106179766237
  (0, 2178)	0.0503729459541
  (0, 26646)	0.377326028937
  (0, 28460)	0.16819436264
  (0, 3688)	0.188663014468
  (0, 8321)	0.232342387504
  (0, 29072)	0.222256468954
  (0, 19291)	0.222315551105
  (0, 14472)	0.172945381572
  (0, 31879)	0.0951766995878
  (0, 5397)	0.116218656405
  (0, 1739)	0.124872299645
  (0, 22491)	0.20767366893
  (0, 10210)	0.188663014468
  (0, 8685)	0.068576941428
  (0, 21516)	0.134181367753
  (0, 25806)	0.0992357073551
  (0, 26651)	0.186380117027
  (0, 14469)	0.138948585788
  (0, 25241)	0.0662660813351
  (0, 6483)	0.0989023438953
  (0, 24937)	0.215532485378
  (0, 6414)	0.215532485378


In [10]:
#load tf info
from collections import defaultdict

words_tf = defaultdict(lambda: 0)
with open("tf.txt") as fin:
    for line in fin:
        tf, word = line.strip().split()
        tf = int(tf)
        
        words_tf[preprocess(word)] += tf
        
f_words_tf = np.fromiter((words_tf[w_m[0]] for w_m in processor.dict), dtype=int)
print(f_words_tf.shape)      

(49674,)


In [11]:
"""
words_tf = None
cache = None

import gc
gc.collect()
"""

'\nwords_tf = None\ncache = None\n\nimport gc\ngc.collect()\n'

In [12]:
BAD_W = 1

from scipy.sparse import vstack, hstack
from random import randint

X = []
Y = []

for i in tqdm(range(f_words.shape[0] // 1)):
    f_w = f_words[i:i+1, :]
    f_m = f_meaning[i:i+1, :]
        
    Y.append(1)
    X.append(hstack([f_w, f_m]))
    for j in range(BAD_W):
        while True:
           rand_i = randint(0, f_words.shape[0] - 1)
           if rand_i != i:
                break
        Y.append(0)    
        X.append(hstack([f_w, f_meaning[rand_i:rand_i+1, :]]))
        
    #break    
X = vstack(X)    
Y = np.array(Y)

100%|██████████| 49674/49674 [00:39<00:00, 1241.87it/s]


In [13]:
print(X.shape)
print(Y.shape)
print(f_words.shape)
print(f_meaning.shape)

(99348, 88063)
(99348,)
(49674, 54649)
(49674, 33414)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [15]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [16]:
t = "svm"

if t == "svm":
    svm = LinearSVC()
    clf = CalibratedClassifierCV(svm) 
    
clf.fit(X_train, y_train)

y_proba = clf.predict_proba(X_test)

In [17]:
from sklearn.metrics import roc_auc_score
print(y_proba)
print(y_test)
print(roc_auc_score(y_test, y_proba[:, 1]))

[[ 0.56529279  0.43470721]
 [ 0.37849615  0.62150385]
 [ 0.2991734   0.7008266 ]
 ..., 
 [ 0.70596609  0.29403391]
 [ 0.44595127  0.55404873]
 [ 0.22994198  0.77005802]]
[1 0 1 ..., 0 1 1]
0.849043919305


In [31]:
y_res = clf.predict_proba(X)[:, 1]#[1::BAD_W + 1]
y_test = Y#[1::BAD_W + 1]

In [32]:
#0.9 0.2 0.4 0.5
#1 1 0 0

freq = np.repeat(f_words_tf.astype(float) / f_words_tf.max(), 2)
freq[np.repeat(f_words_tf, 2) == 0] = 1.0

#freq = 0.0

print(freq)

diff = np.abs(y_res - y_test) - freq
diff_order = np.argsort(diff)[::-1] #diff[diff_order] - decreasing

N = 100

#print(diff[diff_order])
print(diff[diff_order[:N]])
print(y_res[diff_order[:N]])
print(y_test[diff_order[:N]])

docs_nums = diff_order[:N] // (1 + BAD_W)
print(docs_nums)

for i in docs_nums:
    print("%s - %s" % processor.dict[i])

[ 1.  1.  1. ...,  1.  1.  1.]
[ 0.8731952   0.84965464  0.84664677  0.84616349  0.84576961  0.84406382
  0.8437697   0.8346675   0.83378408  0.8336821   0.82767698  0.82608909
  0.82462002  0.82301877  0.82251048  0.82094878  0.81999296  0.81980004
  0.81788435  0.81679746  0.81563879  0.81415247  0.81410297  0.81385181
  0.81316406  0.81301548  0.81260175  0.81189826  0.81146649  0.81127118
  0.80943599  0.8093374   0.8079505   0.80784919  0.80728945  0.80663182
  0.80442682  0.80428842  0.80357538  0.80356838  0.80352647  0.80290337
  0.80285352  0.80283141  0.80250087  0.80238094  0.8017606   0.80049573
  0.80038434  0.8000525   0.79995721  0.79959852  0.7985339   0.79825488
  0.79809205  0.79709751  0.79669694  0.79655585  0.79654943  0.79642498
  0.79642236  0.79638735  0.79625803  0.79604217  0.79581423  0.79541671
  0.79517597  0.79483183  0.79482602  0.79450488  0.79423231  0.79421576
  0.79411808  0.79351118  0.79290513  0.79270193  0.79264985  0.79246
  0.79239228  0.7922896